In [1]:
apikey = "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"

In [2]:
import json
import getpass
import os
import pandas as pd
import requests
import io
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, SystemMessage, BaseMessage
#add memory
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict
from typing import Sequence

In [3]:
# define langchain requirements and set the model

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = apikey
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]="pr-abandoned-estrogen-14"

from langchain_ollama import ChatOllama

model= ChatOllama(model="llama3.2", temperature=1)

In [4]:
# scrap the occupation file

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:132.0) Gecko/20100101 Firefox/132.0',
}

response = requests.get('https://download.bls.gov/pub/time.series/oe/oe.occupation', headers=headers)
response.status_code

200

In [5]:
# load occupation to df
#https://www.bls.gov/oes/current/oessrci.htm
occupation = pd.read_csv(io.BytesIO(response.content), sep="\t").dropna(subset=["occupation_description"])
occupation = occupation[["occupation_code", "occupation_name","occupation_description"]]
occupation = occupation.rename(columns={"occupation_code":"code", "occupation_name":"name","occupation_description":"description"})


In [6]:
#occupation_l = ['New Accounts Clerks', 'Nuclear Technicians', 'Orderlies', 'Clinical and Counseling Psychologists', 'Radiologic Technologists and Technicians'] 
occupation_l = (list(occupation.sample(n = 5, random_state=123).name))
sample_o = occupation.loc[occupation["name"].isin(occupation_l)]
sample_o

,code,name,description
284,251053,"Environmental Science Teachers, Postsecondary",Teach courses in environmental science. Includ...
322,252021,"Elementary School Teachers, Except Special Edu...",Teach academic and social skills to students a...
576,359011,Dining Room and Cafeteria Attendants and Barte...,Facilitate food service. Clean tables; remove ...
602,392011,Animal Trainers,"Train animals for riding, harness, security, p..."
735,439071,"Office Machine Operators, Except Computer",Operate one or more of a variety of office mac...


In [9]:
s_name = sample_o["name"].iloc[3]
s_des = sample_o["description"].iloc[3]

In [15]:
#creating prompt from jd and name.

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "you are a {name}.this is what you do: {description} Be helpful and concise with your responses. do not leave your character, stay in character always, do not answer things out of your character",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [11]:
# Define a new graph

class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    name: str
    description: str


workflow = StateGraph(state_schema=State)


# Define the function that calls the model
def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)


In [16]:
# setting up agents

for x in range(len(sample_o)):
    name=sample_o.iloc[x]["name"]
    description=sample_o.iloc[x]["description"]
    query = "1 short sentence to describe your work"
    config = {"configurable": {"thread_id": str(x)*3}}
    input_messages = [HumanMessage(query)]
    output = app.invoke({"messages": input_messages, "description": description, "name" : name}, config)
    print(output["messages"][-1].content, config)  # output contains all messages in state

I teach and educate students about the natural world, ecosystems, and environmental issues at a postsecondary level. {'configurable': {'thread_id': '000'}}
As an elementary school teacher, I help young students develop academic and social skills to succeed in school and beyond. {'configurable': {'thread_id': '111'}}
I help serve food and beverages to patrons in the dining area, maintaining a clean and welcoming environment. {'configurable': {'thread_id': '222'}}
I specialize in training animals for specific tasks, behaviors, and roles that enhance their well-being, safety, and ability to assist humans. {'configurable': {'thread_id': '333'}}
I operate office machines like photocopying, photographic, and duplicating machines to help our company produce and distribute documents efficiently. {'configurable': {'thread_id': '444'}}


In [17]:
# set up function to use the agents

def useagent1(query, idnum):

    config = {"configurable": {"thread_id": idnum}}
    input_messages = [HumanMessage(query)]
    output = app.invoke({"messages": input_messages}, config)
    return output["messages"][-1].content  # output contains all messages in state, 

In [20]:
# check they are running
for x in range(len(sample_o)):
    print(useagent1("what did we discuss", idnum=str(x)*3))

We had a quick conversation, Bob. We started by discussing my work as an Environmental Science teacher at a postsecondary level. Then, I provided a brief description of what I do, and later, you asked me to recommend a car that suits your needs.

To recap, it seems like we didn't actually discuss cars initially. You asked me about the type of car you should buy, but I didn't have any prior conversation to build upon. Would you like to start fresh or explore a different topic related to environmental science?
We just started our conversation, Bob! We talked about your interest in buying a car, and I tried to help. That was the extent of our discussion so far. Now, would you like to talk more about school or something else?
We didn't really have a discussion about cars, Bob. We just started chatting, and I said I'm an attendant in a dining room and cafeteria setting, helping serve food and drinks to patrons.
We discussed your question for a car purchase, Bob! But before that, I was telli

In [15]:
mc_system_prompt = """"You are an expert question designer tasked with creating high-quality certification questions to identify true professionals.
        Your goal is to create 10 multiple choice questions that:
        Test Practical Expertise: Focus on real-world scenarios, tools, and challenges specific to the domain.
        Assess Depth of Knowledge: Ensure questions range from fundamental concepts to advanced problem-solving, covering both theory and application.
        Differentiate Skill Levels: Include nuances that distinguish seasoned professionals from novices, such as best practices, edge-case scenarios, or optimized solutions.
        Simulate Professional Situations: Incorporate case studies or situational judgment problems that mimic day-to-day challenges professionals face.
        Reflect Industry Trends: Include questions that are relevant to current tools, techniques, and standards in the field.
        Each question should have a brief context or scenario.
        Structure questions only on advanced levels of difficulty"""

mc_prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            mc_system_prompt,
        ),
        ("user", "create 10 multiple choice questions about {name} with description {description}.")
        MessagesPlaceholder(variable_name="messages"),
    ]
)

TypeError: 'CompiledStateGraph' object is not iterable

In [63]:
raw_output = """I'll create 3 multiple choice questions with varying difficulty levels and provide the answers.

What is the primary function of mitochondria in a cell?
a) Protein synthesis
b) Energy production
c) Cell division
d) Waste removal
Which ancient civilization built the Great Pyramids of Giza?
a) Romans
b) Greeks
c) Babylonians
d) Egyptians
If x² + 5x + 6 = 0, what are the values of x?
a) -2 and -3
b) 2 and 3
c) -2 and 3
d) 2 and -3

Answers:

b) Energy production - Mitochondria are often called the "powerhouse of the cell" because they produce ATP through cellular respiration.
d) Egyptians - The pyramids were built during Egypt's Old Kingdom period, around 2500 BCE.
a) -2 and -3 - You can solve this by factoring: x² + 5x + 6 = (x + 2)(x + 3) = 0, so x = -2 or x = -3"""


# """Here are 3 multiple choice questions on different topics:

# question 1: Which of the following astronomical objects is closest to Earth?
# a) Alpha Centauri
# b) The Moon
# c) Mars
# d) The Sun
# question 2:In the context of computer programming, what does IDE stand for?
# a) Integrated Development Environment
# b) Interactive Design Engine
# c) Internal Data Exchange
# d) Indexed Database Entry
# question 3: The Renaissance period primarily began in which modern-day country?
# a) France
# b) England
# c) Italy
# d) Spain

# Answers:

# b) The Moon
# a) Integrated Development Environment
# c) Italy

# Would you like me to explain any of these answers or generate more questions on a specific topic?"""




In [64]:
from ollama import chat
from pydantic import BaseModel, Field

class QuestionData(BaseModel):
    question_text: str
    options: list[str]

class QuestionOptions(BaseModel):
    questions: dict[int, QuestionData] = Field(
        description="Maps question numbers to question text and options"
    )


response = chat(
  messages=[
    {
      'role': 'user',
      'content': str('for each question, get the question number as integer. then extract the question text and option and put them according to the format.'+raw_output),
    }
  ],
  model='llama3.1',
  format=QuestionOptions.model_json_schema(),
)

structured_quiz = QuestionOptions.model_validate_json(response["message"]["content"])
print(structured_quiz)

questions={1: QuestionData(question_text='What is the primary function of mitochondria in a cell?', options=['a) Protein synthesis', 'b) Energy production', 'c) Cell division', 'd) Waste removal']), 2: QuestionData(question_text='Which ancient civilization built the Great Pyramids of Giza?', options=['a) Romans', 'b) Greeks', 'c) Babylonians', 'd) Egyptians']), 3: QuestionData(question_text='If x² + 5x + 6 = 0, what are the values of x?', options=['a) -2 and -3', 'b) 2 and 3', 'c) -2 and 3', 'd) 2 and -3'])}


In [98]:
a = "apple"
b = ["a", "b", "c"]
a + str(b)

"apple['a', 'b', 'c']"

In [103]:
#decompose mc
for x in range(len(structured_quiz.questions)):
    question = structured_quiz.questions[x+1].question_text
    options = structured_quiz.questions[x+1].options

    message = [("system", "select the best answer. think step by step then provide your answer at the end."), ("user", question+ str(options))]

    response1 = model.invoke(message)
    print(response1.content)

To determine the correct answer, let's evaluate each option:

a) Protein synthesis: This process occurs in the ribosomes within the cell, not directly in the mitochondria. Mitochondria do play a role in regulating protein synthesis, but they are not responsible for it.

b) Energy production: This is the primary function of mitochondria. They generate most of the energy that cells need to function through a process called cellular respiration, which converts glucose into ATP (adenosine triphosphate).

c) Cell division: While mitochondria divide during cell division to ensure each new cell receives sufficient energy-producing machinery, they do not directly participate in the cell division process.

d) Waste removal: Mitochondria do have some mechanisms for removing waste products, but their primary function is not waste removal.

Considering these options, the correct answer is:

**b) Energy production**

This is the primary function of mitochondria, as it enables cells to generate the 

In [100]:
response1.content

'To solve for x in the equation x² + 5x + 6 = 0, we can use factoring or the quadratic formula.\n\nFactoring: The equation factors as (x + 2)(x + 3) = 0. \n\nSetting each factor equal to zero and solving for x gives us:\n\nx + 2 = 0 --> x = -2\n\nx + 3 = 0 --> x = -3\n\nTherefore, the correct answer is:\n\nc) -2 and -3'

In [ ]:
raw_mc =  """I'll create 3 multiple choice questions with varying difficulty levels and provide the answers.

What is the primary function of mitochondria in a cell?
a) Protein synthesis
b) Energy production
c) Cell division
d) Waste removal
Which ancient civilization built the Great Pyramids of Giza?
a) Romans
b) Greeks
c) Babylonians
d) Egyptians
If x² + 5x + 6 = 0, what are the values of x?
a) -2 and -3
b) 2 and 3
c) -2 and 3
d) 2 and -3

Answers:

b) Energy production - Mitochondria are often called the "powerhouse of the cell" because they produce ATP through cellular respiration.
d) Egyptians - The pyramids were built during Egypt's Old Kingdom period, around 2500 BCE.
a) -2 and -3 - You can solve this by factoring: x² + 5x + 6 = (x + 2)(x + 3) = 0, so x = -2 or x = -3"""

# """
# provide your answers.
# 1. Which of the following astronomical objects is closest to Earth?
# a) Alpha Centauri
# b) The Moon
# c) Mars
# d) The Sun
# 2. In the context of computer programming, what does IDE stand for?
# a) Integrated Development Environment
# b) Interactive Design Engine
# c) Internal Data Exchange
# d) Indexed Database Entry
# 3. The Renaissance period primarily began in which modern-day country?
# a) France
# b) England
# c) Italy
# d) Spain"""

message = [("system", "select the best answer."), ("user", )]

response1 = model.invoke(message)
print(response1.content)




It looks like you're working with a data structure called `QuestionData`. Each value in the dictionary represents a question, which is an instance of `QuestionData`.

Based on this limited information, I don't see any specific questions to answer. However, if you'd like me to help with anything related to these questions, such as:

* Extracting information from the dictionary
* Filtering out options
* Resolving the math problem

Please provide more context or specify what you're trying to accomplish!


In [29]:
response1.content

'Here are the answers:\n\n1. b) The Moon\nThe Moon is, on average, about 239,000 miles (384,000 kilometers) away from Earth.\n\n2. a) Integrated Development Environment\nIDE stands for Integrated Development Environment, which refers to software development tools that provide a comprehensive set of integrated applications and services.\n\n3. c) Italy\nThe Renaissance period, characterized by a cultural and intellectual movement in Europe between the 14th and 17th centuries, primarily began in Italy, particularly in cities such as Florence, Venice, and Rome.'

In [101]:
#to make response to structure output

from ollama import chat
from pydantic import BaseModel, Field

class QuestionAnswers(BaseModel):
    # Method 1: Using Dict with basic validation
    answer: dict[int, str] = Field(
        description="Maps question numbers to answer choices (a-d)"
    )
  

response = chat(
  messages=[
    {
      'role': 'user',
      'content': str('put the questions and answers into the format'+response1.content),
    }
  ],
  model='llama3.1',
  format=QuestionAnswers.model_json_schema(),
)

country = QuestionAnswers.model_validate_json(response["message"]["content"])
print(country)

ValidationError: 2 validation errors for QuestionAnswers
answer.value.[key]
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='value', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/int_parsing
answer.question.[key]
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='question', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/int_parsing

In [40]:
response["message"]["content"]

'{ "name": "Canada", "capital": "Ottawa", "languages": ["English", "French"] }'